<a href="https://colab.research.google.com/github/nandu26m/E-Commerce-Food-Delivery-Data-Analytics-Project/blob/main/notebooks/Python_script_to_generate_sample_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pandas faker
!python generate_users.py

python3: can't open file '/content/generate_users.py': [Errno 2] No such file or directory


### *This Python script generates sample records.*

In [8]:
import pandas as pd
import numpy as np
from faker import Faker
import random
import string
from datetime import timedelta, datetime
from google.colab import files  # Comment if not on Colab

fake = Faker()

# Constants for dataset sizes
NUM_USERS = 10000
NUM_RESTAURANTS = 500
NUM_MENU_ITEMS = 5000
NUM_ORDERS = 50000
NUM_ORDER_ITEMS = 120000
NUM_DELIVERY_PERSONS = 200
NUM_DELIVERIES = 48000
NUM_PAYMENTS = 50000
NUM_RATINGS = 25000
NUM_PAYMENT_METHODS = 5
NUM_PROMOTIONS = 50
NUM_ORDER_PROMOTIONS = 8000
NUM_MENU_ITEM_HISTORY = 10000
NUM_DELIVERY_STATUS_HISTORY = 150000
NUM_USER_LOGINS = 80000

# Shared lists for consistency
cities = ['Berlin', 'Munich', 'Hamburg', 'Cologne', 'Düsseldorf', 'Frankfurt', 'Stuttgart']
devices = ['iOS', 'Android', 'Web']
payment_methods_list = ['Credit Card', 'PayPal', 'Apple Pay', 'Google Pay', 'Cash']

cuisines = [
    'Italian', 'Chinese', 'Indian', 'Mexican', 'Japanese', 'Thai', 'French', 'Mediterranean',
    'American', 'Vietnamese', 'Korean', 'Turkish', 'Greek', 'Spanish', 'Lebanese'
]

order_statuses = ['completed', 'cancelled', 'pending', 'failed']

delivery_statuses = ['pending', 'picked_up', 'in_transit', 'delivered', 'failed']

payment_statuses = ['paid', 'pending', 'failed']

promo_codes = [f'PROMO{str(i).zfill(3)}' for i in range(1, NUM_PROMOTIONS + 1)]


# Utility functions
def random_date(start, end):
    """Generate random datetime between start and end"""
    return fake.date_time_between_dates(datetime_start=start, datetime_end=end)


def random_bool(p=0.5):
    return random.random() < p


# --- Generate Users ---
print('Generating users...')
users = []
for user_id in range(1, NUM_USERS + 1):
    name = fake.name()
    email = fake.unique.email()
    signup_date = random_date(datetime(2020, 1, 1), datetime(2023, 6, 1))
    city = random.choice(cities)
    device = random.choice(devices)
    is_loyal = random_bool(0.3)
    users.append([user_id, name, email, signup_date, city, device, is_loyal])

df_users = pd.DataFrame(users, columns=['user_id', 'name', 'email', 'signup_date', 'city', 'device', 'is_loyal'])


# --- Generate Restaurants ---
print('Generating restaurants...')
restaurants = []
for restaurant_id in range(1, NUM_RESTAURANTS + 1):
    name = fake.company() + " " + random.choice(['Bistro', 'Cafe', 'Kitchen', 'House', 'Diner'])
    cuisine = random.choice(cuisines)
    rating = round(random.uniform(2.5, 5.0), 1)
    city = random.choice(cities)
    restaurants.append([restaurant_id, name, cuisine, rating, city])

df_restaurants = pd.DataFrame(restaurants, columns=['restaurant_id', 'name', 'cuisine', 'rating', 'city'])


# --- Generate Payment Methods ---
print('Generating payment methods...')
payment_methods = []
for method_id, method_name in enumerate(payment_methods_list, start=1):
    payment_methods.append([method_id, method_name])

df_payment_methods = pd.DataFrame(payment_methods, columns=['method_id', 'method_name'])


# --- Generate Menu Items ---
print('Generating menu_items...')
menu_items = []
for item_id in range(1, NUM_MENU_ITEMS + 1):
    restaurant_id = random.randint(1, NUM_RESTAURANTS)
    item_name = fake.word().capitalize() + " " + random.choice(['Pizza', 'Burger', 'Roll', 'Salad', 'Soup', 'Pasta', 'Sushi', 'Sandwich'])
    price = round(random.uniform(5.0, 50.0), 2)
    is_available = random_bool(0.9)
    menu_items.append([item_id, restaurant_id, item_name, price, is_available])

df_menu_items = pd.DataFrame(menu_items, columns=['item_id', 'restaurant_id', 'item_name', 'price', 'is_available'])


# --- Generate Delivery Persons ---
print('Generating delivery_persons...')
delivery_persons = []
for delivery_person_id in range(1, NUM_DELIVERY_PERSONS + 1):
    name = fake.name()
    phone = fake.phone_number()
    city = random.choice(cities)
    active_status = random_bool(0.95)
    delivery_persons.append([delivery_person_id, name, phone, city, active_status])

df_delivery_persons = pd.DataFrame(delivery_persons, columns=['delivery_person_id', 'name', 'phone', 'city', 'active_status'])


# --- Generate Orders ---
print('Generating orders...')
orders = []
for order_id in range(1, NUM_ORDERS + 1):
    user_id = random.randint(1, NUM_USERS)
    # Assign restaurant from those in the same city as user for realism
    user_city = df_users.loc[df_users.user_id == user_id, 'city'].values[0]
    rest_candidates = df_restaurants[df_restaurants.city == user_city]
    if len(rest_candidates) == 0:
        # fallback if no restaurants in user city
        restaurant_id = random.randint(1, NUM_RESTAURANTS)
    else:
        restaurant_id = rest_candidates.sample(1).restaurant_id.values[0]
    order_datetime = random_date(datetime(2022, 1, 1), datetime(2023, 6, 30))
    payment_method_id = random.randint(1, NUM_PAYMENT_METHODS)
    status = random.choices(order_statuses, weights=[0.7, 0.1, 0.15, 0.05])[0]

    # cancellation fields
    canceled_at = None
    cancellation_reason = None
    if status == 'cancelled':
        canceled_at = order_datetime + timedelta(minutes=random.randint(5, 120))
        cancellation_reason = random.choice(['Customer Request', 'Restaurant Closed', 'Payment Failure', 'Out of Delivery Area'])

    created_at = order_datetime - timedelta(minutes=random.randint(10, 1440))
    updated_at = order_datetime + timedelta(minutes=random.randint(0, 1440))

    # total_amount is sum of order_items but unknown here, placeholder 0
    total_amount = 0

    orders.append([order_id, user_id, restaurant_id, order_datetime, total_amount, payment_method_id, status, canceled_at, cancellation_reason, created_at, updated_at])

df_orders = pd.DataFrame(orders, columns=['order_id', 'user_id', 'restaurant_id', 'order_datetime', 'total_amount', 'payment_method_id', 'status', 'canceled_at', 'cancellation_reason', 'created_at', 'updated_at'])


# --- Generate Order Items ---
print('Generating order_items...')
order_items = []
order_item_id = 1
for order_id in range(1, NUM_ORDERS + 1):
    num_items = np.random.poisson(2.4)
    if num_items < 1:
        num_items = 1
    chosen_items = df_menu_items.sample(num_items)
    total_order_amount = 0
    for _, row in chosen_items.iterrows():
        quantity = random.randint(1, 3)
        item_price = row['price']
        total_order_amount += item_price * quantity
        order_items.append([order_item_id, order_id, row['item_id'], quantity, item_price])
        order_item_id += 1

    # Update total_amount in orders
    df_orders.loc[df_orders.order_id == order_id, 'total_amount'] = round(total_order_amount, 2)

df_order_items = pd.DataFrame(order_items, columns=['order_item_id', 'order_id', 'item_id', 'quantity', 'item_price'])


# --- Generate Deliveries ---
print('Generating deliveries...')
deliveries = []
for delivery_id in range(1, NUM_DELIVERIES + 1):
    order_row = df_orders.sample(1).iloc[0]
    order_id = order_row['order_id']

    # Only for completed or pending orders
    if order_row['status'] not in ['completed', 'pending']:
        # To keep counts, we'll still create delivery, but mark status accordingly
        status = 'failed'
    else:
        status = random.choices(delivery_statuses, weights=[0.1, 0.15, 0.3, 0.4, 0.05])[0]

    # Choose delivery person in same city as restaurant if possible
    restaurant_city = df_restaurants.loc[df_restaurants.restaurant_id == order_row['restaurant_id'], 'city'].values[0]
    dp_candidates = df_delivery_persons[df_delivery_persons.city == restaurant_city]
    if len(dp_candidates) == 0:
        delivery_person_id = random.randint(1, NUM_DELIVERY_PERSONS)
    else:
        delivery_person_id = dp_candidates.sample(1).delivery_person_id.values[0]

    pickup_time = order_row['order_datetime'] + timedelta(minutes=random.randint(10, 30))
    drop_time = pickup_time + timedelta(minutes=random.randint(10, 60))
    distance_km = round(random.uniform(0.5, 15.0), 2)
    created_at = pickup_time - timedelta(minutes=random.randint(10, 30))
    updated_at = drop_time + timedelta(minutes=random.randint(0, 30))

    deliveries.append([delivery_id, order_id, delivery_person_id, pickup_time, drop_time, distance_km, status, created_at, updated_at])

df_deliveries = pd.DataFrame(deliveries, columns=['delivery_id', 'order_id', 'delivery_person_id', 'pickup_time', 'drop_time', 'distance_km', 'status', 'created_at', 'updated_at'])


# --- Generate Delivery Status History ---
print('Generating delivery_status_history...')
delivery_status_history = []
status_id = 1
for delivery_id in range(1, NUM_DELIVERIES + 1):
    num_updates = random.randint(2, 4)
    base_time = df_deliveries.loc[df_deliveries.delivery_id == delivery_id, 'created_at'].iloc[0].to_pydatetime()
    for _ in range(num_updates):
        status = random.choice(delivery_statuses)
        timestamp = base_time + timedelta(minutes=random.randint(5, 120))
        delivery_status_history.append([status_id, delivery_id, status, timestamp])
        status_id += 1
        base_time = timestamp

df_delivery_status_history = pd.DataFrame(delivery_status_history, columns=['status_id', 'delivery_id', 'status', 'timestamp'])


# --- Generate Payments ---
print('Generating payments...')
payments = []
for payment_id in range(1, NUM_PAYMENTS + 1):
    order_row = df_orders.iloc[payment_id - 1]
    amount = order_row['total_amount']
    payment_method_id = order_row['payment_method_id']
    payment_datetime = order_row['order_datetime'] + timedelta(minutes=random.randint(1, 60))
    payment_status = random.choices(payment_statuses, weights=[0.85, 0.1, 0.05])[0]
    created_at = payment_datetime - timedelta(minutes=random.randint(1, 10))
    updated_at = payment_datetime + timedelta(minutes=random.randint(1, 10))
    payments.append([payment_id, order_row['order_id'], amount, payment_method_id, payment_datetime, payment_status, created_at, updated_at])

df_payments = pd.DataFrame(payments, columns=['payment_id', 'order_id', 'amount', 'payment_method_id', 'payment_datetime', 'payment_status', 'created_at', 'updated_at'])


# --- Generate Ratings ---
print('Generating ratings...')
ratings = []
rated_order_ids = random.sample(list(df_orders['order_id']), NUM_RATINGS)
for rating_id, order_id in enumerate(rated_order_ids, start=1):
    order_row = df_orders.loc[df_orders.order_id == order_id].iloc[0]
    user_id = order_row['user_id']
    restaurant_id = order_row['restaurant_id']
    rating_val = round(random.uniform(1, 5), 1)
    feedback = fake.sentence(nb_words=12)
    created_at = order_row['order_datetime'] + timedelta(days=random.randint(0, 7))
    updated_at = created_at + timedelta(days=random.randint(0, 5))
    ratings.append([rating_id, order_id, user_id, restaurant_id, rating_val, feedback, created_at, updated_at])

df_ratings = pd.DataFrame(ratings, columns=['rating_id', 'order_id', 'user_id', 'restaurant_id', 'rating', 'feedback', 'created_at', 'updated_at'])


# --- Generate Promotions ---
print('Generating promotions...')
promotions = []
for promo_id in range(1, NUM_PROMOTIONS + 1):
    promo_code = promo_codes[promo_id - 1]
    discount_percent = round(random.uniform(5, 50), 2)
    start_date = random_date(datetime(2023, 1, 1), datetime(2023, 6, 1))
    end_date = start_date + timedelta(days=random.randint(10, 90))
    promotions.append([promo_id, promo_code, discount_percent, start_date, end_date])

df_promotions = pd.DataFrame(promotions, columns=['promo_id', 'promo_code', 'discount_percent', 'start_date', 'end_date'])


# --- Generate Order Promotions ---
print('Generating order_promotions...')
order_promotions = []
promo_order_ids = random.sample(list(df_orders['order_id']), NUM_ORDER_PROMOTIONS)
for op_id, order_id in enumerate(promo_order_ids, start=1):
    promo_id = random.randint(1, NUM_PROMOTIONS)
    # Discount amount roughly proportional to discount_percent * total_amount
    order_amount = df_orders.loc[df_orders.order_id == order_id, 'total_amount'].values[0]
    discount_percent = df_promotions.loc[df_promotions.promo_id == promo_id, 'discount_percent'].values[0]
    discount_amount = round(order_amount * (discount_percent / 100), 2)
    order_promotions.append([op_id, order_id, promo_id, discount_amount])

df_order_promotions = pd.DataFrame(order_promotions, columns=['id', 'order_id', 'promo_id', 'discount_amount'])


# --- Generate Menu Item History ---
print('Generating menu_item_history...')
menu_item_history = []
for history_id in range(1, NUM_MENU_ITEM_HISTORY + 1):
    item_id = random.randint(1, NUM_MENU_ITEMS)
    price = round(random.uniform(5.0, 50.0), 2)
    is_available = random_bool(0.9)
    updated_at = random_date(datetime(2022, 1, 1), datetime(2023, 6, 1))
    menu_item_history.append([history_id, item_id, price, is_available, updated_at])

df_menu_item_history = pd.DataFrame(menu_item_history, columns=['history_id', 'item_id', 'price', 'is_available', 'updated_at'])


# --- Generate User Logins ---
print('Generating user_logins...')
user_logins = []
for login_id in range(1, NUM_USER_LOGINS + 1):
    user_id = random.randint(1, NUM_USERS)
    login_time = random_date(datetime(2022, 1, 1), datetime(2023, 6, 30))
    device = random.choice(devices)
    ip_address = fake.ipv4()
    user_logins.append([login_id, user_id, login_time, device, ip_address])

df_user_logins = pd.DataFrame(user_logins, columns=['login_id', 'user_id', 'login_time', 'device', 'ip_address'])


# --- Save all dataframes as CSV ---
print('Saving CSV files...')
dfs = {
    'users.csv': df_users,
    'restaurants.csv': df_restaurants,
    'menu_items.csv': df_menu_items,
    'orders.csv': df_orders,
    'order_items.csv': df_order_items,
    'delivery_persons.csv': df_delivery_persons,
    'deliveries.csv': df_deliveries,
    'delivery_status_history.csv': df_delivery_status_history,
    'payment_methods.csv': df_payment_methods,
    'payments.csv': df_payments,
    'ratings.csv': df_ratings,
    'promotions.csv': df_promotions,
    'order_promotions.csv': df_order_promotions,
    'menu_item_history.csv': df_menu_item_history,
    'user_logins.csv': df_user_logins,
}

for filename, df in dfs.items():
    df.to_csv(filename, index=False)

print('All CSV files generated.')

Generating users...
Generating restaurants...
Generating payment methods...
Generating menu_items...
Generating delivery_persons...
Generating orders...
Generating order_items...


/tmp/ipython-input-8-2117063409.py:173: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '38.1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_orders.loc[df_orders.order_id == order_id, 'total_amount'] = round(total_order_amount, 2)


Generating deliveries...
Generating delivery_status_history...
Generating payments...
Generating ratings...
Generating promotions...
Generating order_promotions...
Generating menu_item_history...
Generating user_logins...
Saving CSV files...
All CSV files generated.


In [9]:
# Uncomment below to download files in Colab one by one:
for filename in dfs.keys():
    files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>